In [9]:
!pip install nemo_toolkit[asr] torchaudio pydub transformers sentencepiece gradio
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
  Using cached pybind11-3.0.0-py3-none-any.whl.metadata (10.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [1]:
import nemo.collections.asr as nemo_asr

asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")

stt_en_conformer_transducer_xlarge.nemo:   0%|          | 0.00/2.58G [00:00<?, ?B/s]

[NeMo I 2025-07-15 13:58:07 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-15 13:58:08 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket1/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket2/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket3/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket4/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket5/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket6/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket7/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket8/tarred_audio_manifest.json
    sample_rate: 16000

[NeMo I 2025-07-15 13:58:08 nemo_logging:393] PADDING: 0
[NeMo I 2025-07-15 13:58:15 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 13:58:15 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 13:58:15 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 13:58:28 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_transducer_xlarge/snapshots/9a8498bb1d32eedb04a2d23f4da076879ba4392a/stt_en_conformer_transducer_xlarge.nemo.


In [8]:
import torchaudio
import time
from pydub import AudioSegment
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import os
import nemo.collections.asr as nemo_asr

# Load translation model
def load_translation_model(target_lang="hi"):
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    tokenizer.src_lang = "en"
    return model, tokenizer, target_lang

# Load ASR model
asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")

# Streaming + context-aware translation
def simulate_streaming_context_aware(audio_path, chunk_duration_ms=2000, target_lang="hi"):
    print(f"🔄 Loading audio: {audio_path}")

    # Auto-detect file format using pydub
    ext = os.path.splitext(audio_path)[1].lower()
    if ext not in ['.mp3', '.wav']:
        raise ValueError("Unsupported audio format. Please provide a .mp3 or .wav file.")

    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_frame_rate(16000)  # NeMo ASR prefers 16kHz
    audio = audio.set_sample_width(2)    # 16-bit PCM
    audio = audio.set_channels(1)        # mono

    chunks = [audio[i:i + chunk_duration_ms] for i in range(0, len(audio), chunk_duration_ms)]

    total_start = time.time()
    model, tokenizer, tgt_lang = load_translation_model(target_lang)

    full_transcription = ""

    for i, chunk in enumerate(chunks):
        chunk_fp = f"chunk_{i}.wav"
        chunk.export(chunk_fp, format="wav")

        start = time.time()

        try:
            transcription_result = asr_model.transcribe([chunk_fp])[0]
            # Access the text attribute and then strip
            transcription = transcription_result.text
        except Exception as e:
            print(f"⚠️ ASR error on chunk {i}: {e}")
            transcription = ""

        full_transcription += transcription.strip() + " "

        end = time.time()
        print(f"[Chunk {i}] ➤ {transcription.strip()} (Latency: {round(end - start, 2)}s)")

        os.remove(chunk_fp)  # Clean up

    print(f"\n📜 Full Transcription:\n{full_transcription.strip()}")

    # Translate entire transcription
    translation_start = time.time()
    inputs = tokenizer(full_transcription.strip(), return_tensors="pt")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    translation_end = time.time()

    print(f"\n🌍 Translated Text ({tgt_lang}):\n{translated_text}")
    print(f"\n🕒 STT+Translation Total Latency: {round(time.time() - total_start, 2)}s")
    print(f"🕒 Translation Latency: {round(translation_end - translation_start, 2)}s")

[NeMo I 2025-07-15 14:20:12 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-15 14:20:12 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket1/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket2/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket3/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket4/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket5/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket6/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket7/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket8/tarred_audio_manifest.json
    sample_rate: 16000

[NeMo I 2025-07-15 14:20:12 nemo_logging:393] PADDING: 0
[NeMo I 2025-07-15 14:20:18 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 14:20:18 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 14:20:18 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-15 14:20:31 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_transducer_xlarge/snapshots/9a8498bb1d32eedb04a2d23f4da076879ba4392a/stt_en_conformer_transducer_xlarge.nemo.


In [9]:
simulate_streaming_context_aware("/content/part1.mp3", chunk_duration_ms=2000, target_lang="hi")  # Hindi

🔄 Loading audio: /content/part1.mp3


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


[Chunk 0] ➤ with the very latest thirty percent (Latency: 0.76s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]


[Chunk 1] ➤ tariffs on the e u and mexico (Latency: 0.12s)


...

[Chunk 32] ➤ a privilege of selling their products in (Latency: 0.08s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00, 16.59it/s]


[Chunk 33] ➤ to us consumers and what are we talking about (Latency: 0.08s)


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]


⚠️ ASR error on chunk 34: Argument #4: Padding size should be less than the corresponding input dimension, but got: padding (256, 256) at dimension 2 of input [1, 1, 176]
[Chunk 34] ➤  (Latency: 0.12s)

📜 Full Transcription:
with the very latest thirty percent tariffs on the e u and mexico announced by the trumpet ministration we have additional on other countries last week including japan south korea canada and brazil plus the fifty percent tariff on all copper imports going to get in to the importance of dr copper in just a couple of minutes also twenty five percent tariffs south korea and japan a potential thirty five percent blanket tariff on canadian goods we thought the negotiations underway there those look very strong at the moment but a also we have massive tariff and blanket tariffs potentially against countries we haven't even heard from we haven't heard a lot from china lately that's obviously a big trade partner when all comes down to is what are we importing into this cou